### Cleaning: GDP and Gov Exp model

Author: Cornelia Ilin <br>
Email: cilin@ischool.berkeley.edu <br>
Date created: November 25, 2020


Citations: <br>

   - GDP data (nominal value, current prices, non-seasonally adjusted, national currency): 
    https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1409151240976
    
   - GDP components data (nominal value, current prices, non-seasonally adjusted, national currency)
   https://data.imf.org/regular.aspx?key=61545852
   
   

#### Step 1: Import packages

In [1]:
# standard
import pandas as pd
import numpy as np
import os

# matching
import fuzzymatcher

# plotting
import matplotlib.pyplot as plt

# analysis
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import sklearn.preprocessing as sklp

# image
from PIL import Image

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

#### Step 2: Define working directories

In [2]:
# in dirs
in_cases = '/Users/cilin/Dropbox/GPL_CDC_covid/data/int/epi/'
in_poli = '/Users/cilin/Dropbox/GPL_CDC_covid/data/int/policy/'
in_gdp_w2020 = '/Users/cilin/Dropbox/GPL_CDC_covid/data/raw/gdp_components/with_2020_data/'
in_mob = '/Users/cilin/Dropbox/GPL_CDC_covid/data/raw/mobility/'
in_er = '/Users/cilin/Dropbox/GPL_CDC_covid/data/raw/exchange_rates/'

# out dirs
out = '/Users/cilin/Dropbox/GPL_CDC_covid/data/reg/gdp/w_2020_data/'

#### Step 3: Define functipops

``match gdp, debt and policy country names``

In [3]:
def read_clean_gdp_match():
    ''' Read and clean gdp data for country name matching
    params:
    -------
    None
    
    return:
    -------
    df, cleaned gdp data for matching
    
    '''
    # create empty df that will hold data for all countries
    df0 = pd.DataFrame()

    for file in os.listdir(in_gdp_w2020):
        if file.endswith('.xlsx'):
            # save country name
            temp_iso2 = file[19:-5]

            # read data
            temp_df = pd.read_excel(in_gdp_w2020 + file,  
                       sheet_name='Quarterly', skiprows=7, header=0)

            # add country column
            temp_df['country'] = temp_iso2
            
            # keep only country column
            cols = 'country'
            temp_df = temp_df[cols]

            df0 = pd.concat([df0, temp_df], axis=0)
            
    # drop duplicates and reset index
    df0.drop_duplicates(inplace=True)
    df0.reset_index(drop=True, inplace=True)
    df0.columns = ['country']
    
    return df0

In [4]:
def read_clean_poli_match():
    ''' Read and clean policy data for country name matching
    params:
    -------
    None
    
    return:
    -------
    df, cleaned policy data for matching
    
    '''
    # read #
    ########
    df = pd.read_csv(in_poli + 'adm0_policy_popwt.csv', encoding='latin-1')
    
    # clean #
    #########
    # keep only country, iso2
    cols = ['who_region', 'country', 'iso2']
    df = df[cols]
    
    # drop duplicates and reset index
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [5]:
def match_countries(df1, df2):
    ''' Match country names in poli and gdp data
    param:
    ------
    df1: gdp or debt data
    df2: poli data
    
    return:
    -------
    df1, df2 matched country names and iso2
    '''
    # use fuzzy matching for country names
    matches = fuzzymatcher.fuzzy_left_join(df1, df2, ['country'], ['country'])
    
    # drop na
    matches.dropna(inplace=True)

    # keep only cols of interest
    matches = matches[['who_region', 'country_left', 'country_right', 'iso2']]
    
    # rename columns
    matches.rename(columns={'country_left': 'country', 'country_right': 'country_poli'}, inplace=True)
    
    return matches

``read data``

In [6]:
def read_gdp():
    '''
    '''
    # create empty df that will hold data for all countries
    df0 = pd.DataFrame()

    for file in os.listdir(in_gdp_w2020):
        if file.endswith('.xlsx'):
            # save country name
            temp_iso2 = file[19:-5]

            # read data
            temp_df = pd.read_excel(in_gdp_w2020 + file,  
                       sheet_name='Quarterly', skiprows=7, header=0)

            # add country column
            temp_df['country'] = temp_iso2

            # concatenate (add to df0)
            df0 = pd.concat([df0, temp_df], axis=0)

    return df0

In [7]:
def read_cases():
    '''
    '''
    #return pd.read_csv(in_cases + 'adm0_cases.csv')
    return pd.read_csv(in_cases + 'adm0_cases.csv')

In [8]:
def read_poli():
    '''
    '''
    return pd.read_csv(in_poli + 'adm0_policy_popwt.csv', encoding='latin-1')

In [9]:
def read_mob():
    '''
    '''
    return pd.read_csv(in_mob + 'Google_Global_Mobility_Report.csv')

In [10]:
def read_er():
    '''
    '''
    return pd.read_csv(in_er + 'countries_w_2020GDP.csv')

``clean data``

In [11]:
def clean_gdp(df):
    ''' Clean gdp data for analysis
    params:
    -------
    df: gdp data

    return:
    -------
    df, cleaned gdp data
    '''
    
    return gdp_replace_add_drop(df)

In [12]:
def gdp_replace_add_drop(df):
    '''
    '''
    # add iso2 column
    df = df.merge(matches_ngdp, on=['country'], how='right')

    # keep if indicator has specific value:
    gdp_components = ['Gross Domestic Product, Nominal, Domestic Currency']
    df = df[df.Indicator.isin(gdp_components)]


    # extract column names for year*quarters
    non_year_quarter_vars = ('Unnamed', 'Scale', 'Base', 'Indicator', 
                             'who_region', 'country', 'country_poli', 'iso2')
    year_quarter_cols = [col for col in df.columns if not col.startswith(non_year_quarter_vars)]


    # transform from wide to long #
    ###############################
    # keep only obs for val
    df = df[df.Indicator==gdp_components[0]]

    # transform wide
    df = pd.melt(df, id_vars = ['who_region', 'country_poli', 'iso2'],
                var_name='year_quarter',
                value_vars=year_quarter_cols, 
                value_name= 'GDP')

    # add year column
    df['year'] = df.year_quarter.str.slice(0, 4)

    # rename country_poli
    df.rename(columns={'country_poli': 'country'}, inplace=True)
    
    df.reset_index(drop=True, inplace=True)
    
    return df

In [13]:
def clean_cases(df):
    ''' Clean case data for analysis
    params:
    -------
    df: case data
    
    return:
    -------
    df, cleaned case data
    '''
    
    return cases_replace_add_drop(df)

In [14]:
def cases_replace_add_drop(df):
    '''
    '''
    # add year_quarter information
    df['year_quarter'] = np.where(df.date.le('2020-03-31'), '2020Q1', 
                                  np.where(df.date.le('2020-06-30'), '2020Q2', 
                                           np.where(df.date.le('2020-09-30'), '2020Q3', '2020Q4')))
    
    # keep only cols of interest
    cols = ['iso2', 'date', 'year_quarter', 'new_cases_imputed']
    df = df[cols]
    
    # keep if date in 2020Q1-2020Q2, (i.e., eliminate Q3 because policy data is until July 31)
    df = df[df.date.ge('2020-01-01') & (df.date.le('2020-06-30'))] 
    
    # transform df at the year_quarter level instead of date. Sum cases
    df = df.groupby(['iso2', 'year_quarter'], as_index=False)['new_cases_imputed'].sum()
    
    df.reset_index(drop=True, inplace=True)
    
    return df

In [15]:
def clean_poli(df):
    ''' Clean policy data for analysis
    params:
    -------
    df: poli data in long form at the date level
    
    return:
    -------
    df, cleaned policy data in wide form at the year_quarter level
    
    '''
    return poli_replace_add_drop(poli_to_wide(df))

In [16]:
def poli_to_wide(df):
    ''' Transform poli data from long to wide
    param:
    ------
    df: poli data in long form
    
    return:
    ------
    df, poli data in wide form
    '''
    # transform data from long to wide
    df0 = pd.DataFrame()

    for column_value in ['policy_on_popwt', 'policy_off_popwt']:

        # transform wide
        temp_df = pd.pivot(df, index = ['who_region', "country", 'iso2','date', 'nat_pop'], 
                           columns = "policy", values= column_value)

        # rename columns
        column_names = []
        temp_column_names = temp_df.columns
        
        for column in temp_column_names:
            column_names.append(column + column_value[6:])
        temp_df.columns = column_names

        # reset index
        temp_df.reset_index(drop=False, inplace=True)

        # append to df0
        if column_value != 'policy_off_popwt':
            df0 = pd.concat([df0, temp_df], axis=1)
        else:
            cols = [cols for cols in temp_df if cols.startswith('p_')]
            temp_df = temp_df[cols]
            df0 = pd.concat([df0, temp_df], axis=1)
    df0.reset_index(drop=True, inplace=True)   
    
    return df0

In [17]:
def poli_replace_add_drop(df):
    '''
    '''
    ### transform date to datetime
    df['date'] = pd.to_datetime(df.date)
    
    ### replace nan by 0.0
    df.replace(np.nan, 0.0, inplace=True)
    
    ### add year_quarter 
    df['year_quarter'] = np.where(df.date.le('2020-03-31'), '2020Q1', 
                                  np.where(df.date.le('2020-06-30'), '2020Q2', 
                                           np.where(df.date.le('2020-09-30'), '2020Q3', '2020Q4')))
    
    ### add days in a year_quarter 
    df['year_quarter_days'] = np.where(df.year_quarter=='2020Q1', 91, 
                                       np.where(df.year_quarter=='2020Q2', 91, 92))
    
    ### keep if date in 2020Q1-2020Q2, (i.e., eliminate Q3 because policy data is until July 31)
    df = df[df.date.ge('2020-01-01') & (df.date.le('2020-06-30'))] 
    
    ### add year_quarter and pop weighted policies; drop pop weighted policies
    # define initial data to append to it (keep only main cols)
    df0 = df.loc[:, ['who_region', 'country', 'iso2', 'year_quarter', 'nat_pop']]
    
    # drop duplicates (this will transform df0 at the year_quarter level instead of date)
    df0.drop_duplicates(inplace=True)

    # define pop weighted policy columns to weight by year_quarter
    policy_cols = [col for col in df.columns if col.startswith('p')]

    for col in policy_cols:
        # define grouping
        group = ['iso2', 'year_quarter', 'year_quarter_days', col]

        # compute days in year_quarter for each value of policy
        temp_df = df.groupby(group, as_index=False).apply(lambda grp: grp.assign(
                             days_in_q = (grp.date.max()-grp.date.min())))
        
        temp_df['days_in_q'] = temp_df.days_in_q.dt.days + 1
        
        # compute share in year_quarter for each value of policy
        temp_df['share_in_q'] = temp_df.days_in_q/temp_df.year_quarter_days
        
        # multiply share in year_quarter by population weighted policy
        temp_df[col + '_qwt'] = temp_df.share_in_q * temp_df[col]

        # keep only cols to be appended to df0
        temp_df = temp_df[['iso2', 'date', 'year_quarter', col + '_qwt']]

        # transform temp_df at the year_quarter level instead of date
        temp_df.drop_duplicates(subset=['iso2', 'year_quarter', col + '_qwt'], inplace=True)
        temp_df = temp_df.groupby(['iso2', 'year_quarter'], as_index=False)[col + '_qwt'].sum()

        # append temp_df to df0
        df0 = df0.merge(temp_df, on=['iso2', 'year_quarter'], how='left')
        
    # add grouped policies
    df0['p_travrest_grp_mand_on_popwt_qwt'] = (df0.p_travrest_flght_mand_on_popwt_qwt + 
                                                df0.p_travrest_tradv_mand_on_popwt_qwt + 
                                                df0.p_travrest_trban_mand_on_popwt_qwt+
                                                df0.p_travrest_airprt_mand_on_popwt_qwt)/4
    
    df0.reset_index(drop=True, inplace=True)
        
    return df0

In [18]:
def clean_mob(df):
    '''
    '''
    
    # keep if sub_region is NAN (this will only return country data)
    df = df[(df.sub_region_1.isna()) & (df.sub_region_2.isna()) & (df.metro_area.isna())]
    
    # rename columns
    df.rename(columns={'country_region_code': 'iso2', 'country_region': 'country',
                       'residential_percent_change_from_baseline': 'mob_residential'}, 
                       inplace=True)
    
    # keep only cols of interest
    cols = ['iso2', 'country', 'date', 'mob_residential']
    df = df[cols]
    
    # transf mob_resid to pp
    df['mob_residential'] = df.mob_residential/100
    
    # add year_quarter information
    df['year_quarter'] = np.where(df.date.le('2020-03-31'), '2020Q1', 
                                  np.where(df.date.le('2020-06-30'), '2020Q2', 
                                           np.where(df.date.le('2020-09-30'), '2020Q3', '2020Q4')))
    
    # keep only cols of interest
    cols = ['iso2', 'country', 'year_quarter', 'date', 'mob_residential']
    df = df[cols]
    
    
    # compute averages in a quarter
    df = df.groupby(['iso2', 'year_quarter'], as_index=False)['mob_residential'].mean()
    
    df.reset_index(drop=True, inplace=True)
    
    return df

``merge data``

In [19]:
def merge_data(df1, df2, df3, df4, df5):
    ''' Merge all data sources
    param:
    ------
    df1: ngdp data
    df2: mob data
    df3: cases data
    df4: poli data
    df5: exchange rate data
    ''' 
    # add exhange rate to gdp
    df1 = df1.merge(df5, on='country', how='left')
    
    # add mob to cases data
    df23 = df3.merge(df2, on=['iso2', 'year_quarter'], how='left')

   
    # add mob_cases to poli data
    df234 = df4.merge(df23, on=['iso2', 'year_quarter'], how='left')


    # add mob_cases_poli to gdp data
    df = df234.merge(df1, on=['country', 'iso2', 'year_quarter'], how='right')
    
    # drop who_region_y
    df.drop(columns=['who_region_y'])

    # rename who_region_x
    df.rename(columns={'who_region_x': 'who_region'}, inplace=True)

    # replace 0 value of who_region value
    def helper(grp):
        grp['who_region'] = grp.who_region.unique()[1]
        return grp
    df = df.groupby(['country'], as_index=False).apply(helper)
    
    # rearange columns
    cols = [col for col in df.columns if not col.startswith('p')] + \
           [col for col in df.columns if col.startswith('p')]
    df = df[cols]
    
    # sort data
    df.sort_values(by=['iso2', 'year_quarter'], inplace=True)
    
    # reset index
    df.reset_index(drop=True, inplace=True)

    return df

``clean merged data``

In [20]:
def clean_merged(df):
    ''' Clean merged data for analysis
    params:
    -------
    df: merged data in wide form at the year_quarter level

    return:
    -------
    df, cleaned merged data
    '''
    
    return merged_replace_add_drop(df)

In [21]:
def merged_replace_add_drop(df):
    '''
    '''
    
    # drop if GDP == ...'
    df = df[df.GDP!='...']
    df.reset_index(drop=True, inplace=True)
    
    ### add ###
    ###########
    
    # add year
    df['year'] = df.year_quarter.str[0:4]

    # add quarter
    df['quarter'] = df.year_quarter.str[4:]

    # add country x quarter
    df['iso2_quarter'] = df.iso2 + '_' + df.quarter
    
    # add GDP transformed to 2020USD
    df['GDP_2020USD'] = df.GDP * df.exchange_rate_2020USD

    # add log of GDP_2020USD
    df.replace([-np.inf, +np.inf], np.nan, inplace=True)
    df.dropna(subset=['GDP_2020USD'], inplace=True)
    df['l_GDP_2020USD'] = np.log(df.GDP_2020USD)
    
    # add year trend
    year_map = {label: idx for idx, label in enumerate(df.year.unique())}
    df['year_trend'] = df.year.map(year_map)
    df['year_trend'] = df.year_trend + 1
    
    # add iso2 * year_trend
    df['year_trend'] = df.year_trend.astype(str)
    df['iso2_year_trend'] = df.iso2 + '_' + df.year_trend
    df['year_trend'] = df.year_trend.astype(int)
    
    # add no_mob_data if the country doesn't have mobility data
    def helper(grp):
        if grp.mob_residential.nunique()==0:
            grp['no_mob_data'] = 1
        else:
            grp['no_mob_data'] = 0
        return grp
        
    df = df.groupby('iso2', as_index=False).apply(helper)


    ### drop ###
    ############
    # drop country if it doesn't have GDP in 2020
    def helper(grp):
        if '2020Q2' in grp.year_quarter.unique():
            return grp

    df = df.groupby(['iso2'], as_index=False).apply(helper)
    df.reset_index(drop=True, inplace=True)
   
    # drop if 2020 Q3 and Q4
    #df = df[df.quarter.isin(('Q1', 'Q2'))]
    df = df[~df.year_quarter.isin(('2020Q3', '2020Q4'))]
    
    ### replace ###
    ###############
    # replace NAN values of policies, cases, and mobility  with 0.0
    cols = [col for col in df.columns if col.startswith('new_')]+ \
           [col for col in df.columns if col.startswith('p_')] + \
           [col for col in df.columns if col.startswith('mob')]

    # compute new cases per 100k
    df['new_cases_per100k'] = (df.new_cases_imputed * 100000)/df.nat_pop

    ### rearange ###
    ################
    # rearange columns
    cols = [col for col in df.columns if not col.startswith('p_')] + \
           [col for col in df.columns if col.startswith('p_')]
    df = df[cols]

    for val in cols:
        df[val] = df[val].replace(np.nan, 0.0)
        

    # reset index and sort data
    df.sort_values(by=['iso2','year'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

``plots``

In [22]:
def plots_gdp_components(df):
    '''Plot summary stats of l_GDP 
    params:
    -------
    df with merged data
    
    return:
    -------
    matplotlib graph
    '''
    
    # sort data by year
    df.sort_values(by =['iso2', 'year', 'quarter'], inplace=True)
    
    # define x_date labels and countries of interest
    countries = [val for val in df.country.unique()];

    ### initialize a new plot
    plt.figure(figsize=(30, 200))

    for i in range(0, len(countries)): # change here for how many countries to plot
        # initialize new plot
        plt.subplot(len(countries), 2, i+1)

        # subset data by country
        temp_df = df[df.country==countries[i]]

        # create x_labels for each country
        x_labels = [val for val in temp_df.year_quarter.unique()]
        x_labels.sort()

        # plot time-series #
        ####################
        plt.plot(temp_df.year_quarter, temp_df.l_GDP_2020USD, label='l_GDP_2020USD')

        # add vertical line for 2020Q1 and 2020Q2
        plt.axvline(x='2020Q1', color='gray', label='2020Q1')

        # set legend, title, axis #
        ###########################
        plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
        plt.title(countries[i])
        #plt.xticks(x_labels[::3])
        plt.xticks(x_labels)
        #plt.ylim(5, 24)

    # save figure
    #plt.savefig(out_dir_plots + 'gdp_debt_plots.pdf', bbox_inches = 'tight')  

---
#### Step 4: Match country names
---

In [23]:
# read/clean nominal GDP
match_ngdp = read_clean_gdp_match()

# read/clean policy
match_poli = read_clean_poli_match()

# match nominal GDP and policy
matches_ngdp = match_countries(match_ngdp, match_poli)

del match_ngdp, match_poli

---
#### Step 6: Read and clean gdp, cases, poli data
---

In [24]:
ngdp = clean_gdp(read_gdp())
cases = clean_cases(read_cases())
poli = clean_poli(read_poli())
mob = clean_mob(read_mob())
er = read_er()

---
#### Step 7: Merge gdp, cases, poli data
---

In [39]:
df = clean_merged(merge_data(ngdp, mob, cases, poli, er))
print('Number of countries with GDP and mobility data:', df.country.nunique())
print('Total number of observations:', df.shape[0])

Number of countries with GDP and mobility data: 54
Total number of observations: 1404


In [40]:
print('Example variables values for one country in data...\n')
cols = ['country', 'iso2', 'year_quarter', 'year_trend', 'exchange_rate_2020USD', 'GDP','GDP_2020USD','l_GDP_2020USD', 
        'new_cases_per100k', 'mob_residential', 'p_moverest_trnsp_mand_on_popwt_qwt']

df[df.country=='Romania'][cols]

Example variables values for one country in data...



,country,iso2,year_quarter,year_trend,exchange_rate_2020USD,GDP,GDP_2020USD,l_GDP_2020USD,new_cases_per100k,mob_residential,p_moverest_trnsp_mand_on_popwt_qwt
1144,Romania,RO,2014Q1,1,0.235888,129532.8,30555.276261,10.327293,0.000000,0.000000,0.000000
1145,Romania,RO,2014Q2,1,0.235888,156932.9,37018.640174,10.519177,0.000000,0.000000,0.000000
1146,Romania,RO,2014Q3,1,0.235888,184502.6,43522.010748,10.681022,0.000000,0.000000,0.000000
1147,Romania,RO,2014Q4,1,0.235888,198735.6,46879.409392,10.755334,0.000000,0.000000,0.000000
1148,Romania,RO,2015Q1,2,0.235888,139809.6,32979.453481,10.403640,0.000000,0.000000,0.000000
1149,Romania,RO,2015Q2,2,0.235888,162825.6,38408.659354,10.556038,0.000000,0.000000,0.000000
1150,Romania,RO,2015Q3,2,0.235888,197342.6,46550.816944,10.748300,0.000000,0.000000,0.000000
1151,Romania,RO,2015Q4,2,0.235888,211952.1,49997.027545,10.819719,0.000000,0.000000,0.000000
1152,Romania,RO,2016Q1,3,0.235888,146977.1,34670.183108,10.453635,0.000000,0.000000,0.000000
1153,Romania,RO,2016Q2,3,0.235888,179192.5,42269.420111,10.651819,0.000000,0.000000,0.000000


---
#### Step 8: Plot stats for D_l_GDP and D_l_debt
---

In [27]:
#plots_gdp_components(df)

#### Step 9: Export data

In [28]:
df.to_csv(out + 'gdp_reg_data.csv')